In [1]:
!pip install timm torchvision
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader, random_split
import timm
from timm.loss import LabelSmoothingCrossEntropy
from timm.models.layers import DropPath
import data_preprocessing
from wandb_logger import WandBLogger
from torch.cuda.amp import autocast, GradScaler
import torch.nn as nn
import torch.optim as optim
import os
import random
import numpy as np
from datetime import datetime

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
SEED = 42

In [4]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

In [5]:
def build_optimizer(optimizer_name, model):
    if optimizer_name.lower() == "adamw":
        config = {
            "learning_rate": LR,
            "betas": BETAS,
            "weight_decay": WEIGHT_DECAY
        }
        optimizer = optim.AdamW(
            model.parameters(),
            lr=LR,
            betas=BETAS,
            weight_decay=WEIGHT_DECAY
        )
    elif optimizer_name.lower() == "sgd":
        config = {
            "learning_rate": LR,
            "momentum": MOMENTUM,
            "weight_decay": WEIGHT_DECAY,
            "nesterov": False
        }
        optimizer = optim.SGD(
            model.parameters(),
            lr=LR,
            momentum=MOMENTUM,
            weight_decay=WEIGHT_DECAY,
            nesterov=False
        )
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

    print(f"Optimizer '{optimizer_name}' initialized successfully.")
    return config, optimizer

def build_scheduler(scheduler_name, optimizer):
    if scheduler_name.lower() == "cosineannealinglr + warmup":
        config = {
            "warmup_epochs": WARMUP_EPOCHS,
            "start_factor": START_FACTOR,
            "t_max": T_MAX,
            "milestones": [WARMUP_EPOCHS]
        }
        scheduler = optim.lr_scheduler.SequentialLR(
            optimizer,
            schedulers=[
                optim.lr_scheduler.LinearLR(optimizer, start_factor=START_FACTOR, total_iters=WARMUP_EPOCHS),
                optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_MAX)
            ],
            milestones=[WARMUP_EPOCHS]
        )
    else:
        raise ValueError(f"Unsupported scheduler: {scheduler_name}")

    print(f"Scheduler '{scheduler_name}' initialized successfully.")
    return config, scheduler

In [6]:
!wandb login f8ad3703c9023ee3f86c7242a87d9280b6c031fb

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [7]:
# THE WHOLE TRAINING SETTINGS ARE HERE!

# GENERAL
BATCH_SIZE = 256
EPOCHS = 30
VAL_SPLIT = 0.1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BACKBONE_FREEZING = True

# LOSS
SMOOTHING=0.1
# OPTIMIZER
OPTIMIZER_NAME = "SGD"
LR=0.1
BETAS=(0.9, 0.999)
WEIGHT_DECAY=5e-4
WARMUP_EPOCHS=5
NESTEROV=False
MOMENTUM=0.9
# SCHEDULER
SCHEDULER_NAME = "CosineAnnealingLr + Warmup"
START_FACTOR=1e-6 / 5e-5
T_MAX=EPOCHS - WARMUP_EPOCHS
# Early Stopping
PATIENCE = 6

In [8]:
pipeline = data_preprocessing.CIFAR100Pipeline(val_split=VAL_SPLIT, use_augment=True)
trainset, valset, testset = pipeline.run_pipeline()
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
valloader = DataLoader(valset, batch_size=BATCH_SIZE)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)

100%|██████████| 169M/169M [00:03<00:00, 43.3MB/s]


In [9]:
def create_dino_vit_s16_for_cifar100(freezing=False):
    model = timm.create_model("vit_small_patch16_224_dino", pretrained=True, num_classes=0, drop_path_rate=0.1)

    # Replace the head with CIFAR-100 classification head
    model.head = nn.Linear(model.num_features, 100)

    if freezing:
      # Freeze all parameters except head
      for param in model.parameters():
          param.requires_grad = False

      # Unfreeze only the head
      for param in model.head.parameters():
          param.requires_grad = True

    return model

model = create_dino_vit_s16_for_cifar100(BACKBONE_FREEZING).to(DEVICE)

OPTIMIZER_CONFIG, optimizer = build_optimizer(OPTIMIZER_NAME, model)
SCHEDULER_CONFIG, scheduler = build_scheduler(SCHEDULER_NAME, optimizer)

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name vit_small_patch16_224_dino to current vit_small_patch16_224.dino.
  model = create_fn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

Optimizer 'SGD' initialized successfully.
Scheduler 'CosineAnnealingLr + Warmup' initialized successfully.


In [10]:
print(next(model.parameters()).device)
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / {total:,}")

torch.backends.cudnn.benchmark = True

cuda:0
Trainable params: 38,500 / 21,704,164


In [11]:
run_name = f"CENTRALIZED_{datetime.now().strftime('%Y%m%d-%H%M%S')}"

logger = WandBLogger(
    project_name="federated-learning-project",
    run_name=run_name,
    config={
        # Training Setup
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "device": DEVICE,
        "backbone_freezing": BACKBONE_FREEZING,

        # Loss Configuration
        "label_smoothing": SMOOTHING,

        # Optimizer Configuration
        "optimizer": OPTIMIZER_NAME,
        "optimizer_config": OPTIMIZER_CONFIG,

        # Scheduler Configuration
        "scheduler": SCHEDULER_NAME,
        "scheduler_config": SCHEDULER_CONFIG,

        # Early Stopping
        "patience": PATIENCE
    }
)


wandb: Currently logged in as: s339170 (polito-fl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [12]:
# Loss with label smoothing
criterion = LabelSmoothingCrossEntropy(smoothing=SMOOTHING)

scaler = GradScaler()

best_val_acc = 0.0
epochs_no_improve = 0

# Training loop
for epoch in range(EPOCHS):
    model.train()
    correct, total, train_loss = 0, 0, 0.0

    for x, y in trainloader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()

        with autocast():
            outputs = model(x)
            loss = criterion(outputs, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item() * y.size(0)
        _, pred = torch.max(outputs, 1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    scheduler.step()

    train_acc = correct / total
    train_loss /= total

    logger.log_metrics({
      "train_loss": train_loss,
      "train_acc": train_acc,
      "learning_rate": scheduler.get_last_lr()[0]
    }, step=epoch)


    # Validation
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for x, y in valloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            loss = criterion(outputs, y)

            val_loss += loss.item() * y.size(0)
            _, pred = torch.max(outputs, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    val_acc = correct / total
    val_loss /= total
    logger.log_metrics({
      "val_loss": val_loss,
      "val_acc": val_acc
    }, step=epoch)

    print(f"Epoch {epoch+1:02d}/{EPOCHS} — Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Early stopping logic
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_no_improve = 0
        best_model_state = model.state_dict()  # save best model
        logger.log_model(model, path="best_model.pth")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

model.load_state_dict(best_model_state)

<ipython-input-12-cd02f9d17999>:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-12-cd02f9d17999>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 01/30 — Train Acc: 0.4062 | Val Acc: 0.6162
Epoch 02/30 — Train Acc: 0.5638 | Val Acc: 0.6164
Epoch 03/30 — Train Acc: 0.5245 | Val Acc: 0.5634
Epoch 04/30 — Train Acc: 0.5150 | Val Acc: 0.5738


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 05/30 — Train Acc: 0.5189 | Val Acc: 0.5696
Epoch 06/30 — Train Acc: 0.5214 | Val Acc: 0.5758
Epoch 07/30 — Train Acc: 0.5478 | Val Acc: 0.5598
Epoch 08/30 — Train Acc: 0.5535 | Val Acc: 0.6076
Early stopping triggered at epoch 8


<All keys matched successfully>

In [13]:
#  test
model.eval()
correct, total, test_loss = 0, 0, 0.0
with torch.no_grad():
    for x, y in testloader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        outputs = model(x)
        loss = criterion(outputs, y)

        test_loss += loss.item() * y.size(0)
        _, pred = torch.max(outputs, 1)
        correct += (pred == y).sum().item()
        total += y.size(0)

test_acc = correct / total
test_loss /= total

logger.log_metrics({
    "test_loss": test_loss,
    "test_acc": test_acc
})

logger.finish()

print(f"\n Final Test Accuracy: {test_acc:.4f} | Test Loss: {test_loss:.4f}")


learning_rate,▁▃▅▆████
test_acc,▁
test_loss,▁
train_acc,▁█▆▆▆▆▇█
train_loss,▁▁▃▄▆██▇
val_acc,██▁▃▂▃▁▇
val_loss,▁▁▃▄▆██▇
learning_rate,0.09649
test_acc,0.5979
test_loss,11.97108
train_acc,0.55349



 Final Test Accuracy: 0.5979 | Test Loss: 11.9711
